#### Initial Module importing

In [2]:
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Activation
from keras import losses
from keras.optimizers import Adam, Adagrad, SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import utils
import keras
from keras.layers import BatchNormalization

from tensorflow.keras.datasets import cifar10, cifar100

import numpy as np
import matplotlib.pyplot as plt
import os 

2023-04-11 01:15:32.449184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Random Seed Setting

In [3]:
import random
import tensorflow as tf
import os


def set_seed(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

#### Cifar 10 Data Loading

In [4]:
(cifar10_X_train, cifar10_y_train),(cifar10_X_test, cifar10_y_test)=cifar10.load_data()

print('input_X_train shape: ', cifar10_X_train.shape)
print(cifar10_X_train.shape[0], 'train samples')
print(cifar10_X_test.shape[0], 'test samples')

input_X_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
cifar10_N_CLASSES = len(np.unique(cifar10_y_train))
print(cifar10_N_CLASSES)

10


In [6]:
# output data one-hot encoding 
cifar10_y_train=utils.to_categorical(cifar10_y_train, cifar10_N_CLASSES)
cifar10_y_test=utils.to_categorical(cifar10_y_test, cifar10_N_CLASSES)

# To normalize the value in between 0 and 1 (there are 255 kinds)
cifar10_X_train=cifar10_X_train.astype('float32')
cifar10_X_test=cifar10_X_test.astype('float32')

cifar10_X_train /=255
cifar10_X_test /=255

#### Cifar 100 Loading

In [4]:
(cifar100_X_train, cifar100_y_train),(cifar100_X_test, cifar100_y_test)=cifar100.load_data()

print('input_X_train shape: ', cifar100_X_train.shape)
print(cifar100_X_train.shape[0], 'train samples')
print(cifar100_X_test.shape[0], 'test samples')

input_X_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [98]:
cifar100_N_CLASSES=len(np.unique(cifar100_y_train))
print(cifar100_N_CLASSES)

100


In [ ]:
# output data one-hot encoding 
cifar100_y_train=utils.to_categorical(cifar100_y_train, cifar100_N_CLASSES)
cifar100_y_test=utils.to_categorical(cifar100_y_test, cifar100_N_CLASSES)

# To normalize the value in between 0 and 1 (there are 255 kinds)
cifar100_X_train=cifar100_X_train.astype('float32')
cifar100_X_test=cifar100_X_test.astype('float32')

cifar100_X_train /=255
cifar100_X_test /=255

### Model training - Random Search

In [7]:
params = {
     'learning_rate': (0.1, 0.01,1,0.001 ),
     'epochs': [20,30,40,50],
     'batch_size':[128, 256, 512],
     'dropout': (0, 0.5, 0.8),
     'optimizer': ["Adam","RMSprop","sgd"],
     'decay':(1e-6,1e-4,1e-2),
     'momentum':(0.9,0.5,0.2),
     'l1': (0.01,0.001,0.0001),
     'l2': (0.01,0.001,0.0001),
     'No_of_CONV_and_Maxpool_layers':[2,3],
     'No_of_Dense_Layers': [2,3,4,5],
     'No_of_Units_in_dense_layers':[128,64,256,512],
     'Conv2d_filters':[32,64,128,256],
     'validation_split':[0.1, 0.2],
     'No_of_batch_norm':[True, False]
    }

IMG_CHANNELS = 3
IMAGE_SIZE = cifar10_X_train.shape[1]


In [8]:
def Randomized_Model(learning_rate=0.001, dropout=0.2, optimizer="Adam", decay=0.0,
                      activation="relu", momentum=0.0, l1=0.0, l2=0.0, No_of_batch_norm=False,
                      No_of_CONV_and_Maxpool_layers=3, No_of_Dense_Layers=3, No_of_Units_in_dense_layers=24,
                      Conv2d_filters=60, batch_size=32, epochs=10, validation_split=0.1):

    #model sequential
    model = Sequential()
    
    # input layer
    model.add(Conv2D(Conv2d_filters, (3,3), padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, IMG_CHANNELS)))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    
    # Hidden Layer
    for i in range(0, No_of_CONV_and_Maxpool_layers):
        model.add(Conv2D(Conv2d_filters, (3, 3), padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Dropout(dropout))
        if No_of_batch_norm == True:
            BatchNormalization()
        else:
            pass
        
    
    # Output Layer
    model.add(Flatten())  
    for i in range (0,No_of_Dense_Layers):
        model.add(Dense(units=No_of_Units_in_dense_layers,activation=activation, kernel_regularizer=regularizers.l2(l2),
                  activity_regularizer=regularizers.l1(l1)))
        model.add(Dropout(dropout))
    model.add(Dense(units=512,activation=activation))     
    model.add(Dense(units=cifar10_N_CLASSES, activation='softmax'))
        
    if optimizer == "Adam":
        opt = keras.optimizers.Adam(learning_rate=learning_rate, decay=decay)
    elif optimizer == "RMSprop":
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate,momentum=momentum,decay=decay)
    elif optimizer == "sgd":
        opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay)
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #model.summary()

    return model


In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer

# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=Randomized_Model, epochs=10, batch_size=20, verbose=1)
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=params, 
                          verbose=20,  n_iter=10, n_jobs=1)

grid_result = grid.fit(cifar10_X_train, cifar10_y_train)

/var/folders/0x/n51_7s_n09dfndljqk4rv2zr0000gn/T/ipykernel_81995/3991797947.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=Randomized_Model, epochs=10, batch_size=20, verbose=1)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START Conv2d_filters=256, No_of_CONV_and_Maxpool_layers=3, No_of_Dense_Layers=2, No_of_Units_in_dense_layers=256, No_of_batch_norm=False, batch_size=512, decay=0.01, dropout=0.8, epochs=30, l1=0.01, l2=0.001, learning_rate=0.01, momentum=0.2, optimizer=Adam, validation_split=0.1
Epoch 1/30
59/59 [==============================] - 554s 9s/step - loss: 3.0144 - accuracy: 0.0988 - val_loss: 2.5558 - val_accuracy: 0.1056
Epoch 2/30
59/59 [==============================] - 583s 10s/step - loss: 2.4383 - accuracy: 0.1023 - val_loss: 2.3703 - val_accuracy: 0.1056
Epoch 3/30
59/59 [==============================] - 576s 10s/step - loss: 2.3471 - accuracy: 0.0994 - val_loss: 2.3311 - val_accuracy: 0.1056
Epoch 4/30
59/59 [==============================] - 587s 10s/step - loss: 2.3235 - accuracy: 0.0994 - val_loss: 2.3178 - val_accuracy: 0.1056
Epoch 5/30
59/59 [==============================] - 578s 10s/step - loss: 2.3

In [125]:
model = Randomized_Model()
model.summary()
last_layer_shape = model.layers[-1].output_shape
print("Shape of last layer:", last_layer_shape)

Model: "sequential_847"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2401 (Conv2D)        (None, 32, 32, 128)       3584      
                                                                 
 activation_2323 (Activation  (None, 32, 32, 128)      0         
 )                                                               
                                                                 
 max_pooling2d_2293 (MaxPool  (None, 16, 16, 128)      0         
 ing2D)                                                          
                                                                 
 conv2d_2402 (Conv2D)        (None, 16, 16, 128)       147584    
                                                                 
 activation_2324 (Activation  (None, 16, 16, 128)      0         
 )                                                               
                                                    

In [ ]:
best_params=grid_result.best_params_
best_params